In [5]:
!pip install mediapipe

In [6]:
# STEP 1: Downgrade TensorFlow to 2.15.0 and install compatible TF.js
!pip install --upgrade --quiet tensorflow==2.15.0 tensorflowjs

In [2]:
!pip show keras

Name: keras
Version: 2.15.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: tensorflow


In [3]:
# OPTIONAL: Downgrade tensorflow-decision-forests only if needed
# !pip install tensorflow-decision-forests==1.8.1

# STEP 2: Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras import layers, models, utils
from mediapipe import solutions
from sklearn.metrics import classification_report
from google.colab import drive

# STEP 3: Mount Drive (if needed)
drive.mount('/content/drive')

# Dataset paths
data_train_dir = '/content/drive/MyDrive/SignLanguageDataset/data/train'
data_test_dir = '/content/drive/MyDrive/SignLanguageDataset/data/test'

# Processor class
def get_processor():
    class ASLProcessor:
        def __init__(self):
            self.mp_hands = solutions.hands.Hands(
                static_image_mode=True,
                max_num_hands=1,
                min_detection_confidence=0.7
            )
            self.class_map = {
                'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4,
                'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9,
                'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14,
                'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19,
                'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24,
                'Z': 25, 'space': 26, 'del': 27, 'nothing': 28
            }
            self.reverse_map = {v: k for k, v in self.class_map.items()}

        def load_train_data(self, data_dir):
            X, y = [], []
            for class_name in os.listdir(data_dir):
                class_dir = os.path.join(data_dir, class_name)
                if os.path.isdir(class_dir):
                    for file in os.listdir(class_dir):
                        if file.endswith('.npy'):
                            data = np.load(os.path.join(class_dir, file))
                            if data.shape == (63,):
                                X.append(data)
                                y.append(self.class_map[class_name])
            return np.array(X), np.array(y)

        def process_test_image(self, image_path):
            image = cv2.imread(image_path)
            if image is None:
                return None
            results = self.mp_hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if results.multi_hand_landmarks:
                return np.array([
                    [lm.x, lm.y, lm.z]
                    for hand in results.multi_hand_landmarks
                    for lm in hand.landmark
                ]).flatten()
            return None

        def evaluate_model(self, model, test_dir):
            X_test, y_true = [], []
            for file in os.listdir(test_dir):
                if file.lower().endswith(('.jpg', '.jpeg')):
                    true_label = os.path.splitext(file)[0].lower()
                    if true_label == 'del':
                        true_label = 'del'
                    elif true_label == 'space':
                        true_label = 'space'
                    elif true_label == 'nothing':
                        true_label = 'nothing'
                    else:
                        true_label = true_label.upper()

                    landmarks = self.process_test_image(os.path.join(test_dir, file))
                    if landmarks is not None and landmarks.shape == (63,):
                        X_test.append(landmarks)
                        y_true.append(self.class_map.get(true_label, -1))

            if not X_test:
                raise ValueError("No valid test images found!")

            X_test = np.array(X_test)
            y_true = np.array(y_true)

            valid_idx = y_true != -1
            X_test = X_test[valid_idx]
            y_true = y_true[valid_idx]

            y_pred = np.argmax(model.predict(X_test), axis=1)

            target_names = [self.reverse_map[i] for i in sorted(np.unique(y_true))]
            print("\n=== Test Results ===")
            print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

            return y_true, y_pred

    return ASLProcessor()

# STEP 4: Load and process data
processor = get_processor()
X_train, y_train = processor.load_train_data(data_train_dir)
y_train = utils.to_categorical(y_train, len(processor.class_map))

# STEP 5: Build and train best model (model_deep)
model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(63,)),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(processor.class_map), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=2)

# STEP 6: Evaluate
processor.evaluate_model(model, data_test_dir)

# STEP 7: Convert and save to TensorFlow.js format
output_dir = '/content/drive/MyDrive/SignLanguageDataset/models/tfjs_model'
os.makedirs(output_dir, exist_ok=True)
tfjs.converters.save_keras_model(model, output_dir)
print(f"\n✅ Model converted and saved to {output_dir}")

# OPTIONAL STEP 8: Zip the model directory
!zip -r /content/tfjs_model.zip /content/drive/MyDrive/SignLanguageDataset/models/tfjs_model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/30
1942/1942 - 7s - loss: 0.9449 - accuracy: 0.6996 - 7s/epoch - 4ms/step
Epoch 2/30
1942/1942 - 5s - loss: 0.3229 - accuracy: 0.8954 - 5s/epoch - 2ms/step
Epoch 3/30
1942/1942 - 7s - loss: 0.2486 - accuracy: 0.9203 - 7s/epoch - 4ms/step
Epoch 4/30
1942/1942 - 5s - loss: 0.2178 - accuracy: 0.9280 - 5s/epoch - 3ms/step
Epoch 5/30
1942/1942 - 5s - loss: 0.1994 - accuracy: 0.9372 - 5s/epoch - 3ms/step
Epoch 6/30
1942/1942 - 6s - loss: 0.1865 - accuracy: 0.9388 - 6s/epoch - 3ms/step
Epoch 7/30
1942/1942 - 5s - loss: 0.1767 - accuracy: 0.9434 - 5s/epoch - 3ms/step
Epoch 8/30
1942/1942 - 7s - loss: 0.1716 - accuracy: 0.9453 - 7s/epoch - 4ms/step
Epoch 9/30
1942/1942 - 5s - loss: 0.1628 - accuracy: 0.9463 - 5s/epoch - 2ms/step
Epoch 10/30
1942/1942 - 5s - loss: 0.1575 - accuracy: 0.9497 - 5s/epoch - 3ms/step
Epoch 11/30
1942/1942 - 6s - loss: 0.1538 - accura

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



✅ Model converted and saved to /content/drive/MyDrive/SignLanguageDataset/models/tfjs_model
  adding: content/drive/MyDrive/SignLanguageDataset/models/tfjs_model/ (stored 0%)
  adding: content/drive/MyDrive/SignLanguageDataset/models/tfjs_model/group1-shard1of1.bin (deflated 7%)
  adding: content/drive/MyDrive/SignLanguageDataset/models/tfjs_model/model.json (deflated 79%)
